In [1]:
import os
import numpy as np
import pandas as pd
from glob import glob
from itertools import chain
from PIL import Image
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from torchvision import models


In [2]:
DATA_DIR = r'D:\NIH Xray Dataset'
CSV_PATH = os.path.join(DATA_DIR, 'Data_Entry_2017.csv')
IMG_SIZE = 224

In [4]:

# ========================
# Load and preprocess CSV
# ========================
df = pd.read_csv(CSV_PATH)

# Map image names to file paths
all_image_paths = {os.path.basename(x): x for x in glob(os.path.join(DATA_DIR, '**', '*.png'), recursive=True)}
df['path'] = df['Image Index'].map(all_image_paths.get)

# Extract all labels
df['Finding Labels'] = df['Finding Labels'].map(lambda x: x.replace('No Finding', ''))
all_labels = np.unique(list(chain(*df['Finding Labels'].map(lambda x: x.split('|')).tolist())))
all_labels = [x for x in all_labels if len(x) > 0]

# Create multi-label columns
for c_label in all_labels:
    df[c_label] = df['Finding Labels'].map(lambda finding: 1.0 if c_label in finding else 0.0)

# Multi-label vector
df['disease_vec'] = df.apply(lambda x: x[all_labels].values, axis=1)

# Train-validation split
train_df, valid_df = train_test_split(df,
                                      test_size=0.25,
                                      random_state=2018,
                                      stratify=df['Finding Labels'].map(lambda x: x[:4]))


### print debug
print(f"Total images: {len(df)}")
print(f"Train images: {len(train_df)}")
print(f"Validation images: {len(valid_df)}")
# ========================



Total images: 112120
Train images: 84090
Validation images: 28030


In [7]:
# ========================
# Dataset & DataLoader
# ========================
train_transforms = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),  # MobileNetV2 expects 3 channels
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(5),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)
])

valid_transforms = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)
])

class ChestXrayDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform
        self.paths = df['path'].values
        # Stack the disease vectors into a 2D NumPy array
        self.labels = np.stack(df['disease_vec'].values).astype(np.float32)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_path = self.paths[idx]
        img = Image.open(img_path).convert('RGB')
        if self.transform:
            img = self.transform(img)
        label = torch.tensor(self.labels[idx])
        return img, label


train_dataset = ChestXrayDataset(train_df, transform=train_transforms)
valid_dataset = ChestXrayDataset(valid_df, transform=valid_transforms)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
valid_loader = DataLoader(valid_dataset, batch_size=256, shuffle=False, num_workers=4)

In [8]:
# ========================
# Model: MobileNetV2
# ========================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

model = models.mobilenet_v2(weights=None)
model.classifier[1] = nn.Linear(model.last_channel, len(all_labels))
model = model.to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

Using device: cuda


In [ ]:
# ========================
# Training Loop
# ========================
EPOCHS = 5
for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * images.size(0)

    epoch_loss = running_loss / len(train_dataset)

    # Validation
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for images, labels in valid_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * images.size(0)
    val_loss /= len(valid_dataset)

    print(f"Epoch [{epoch+1}/{EPOCHS}] Train Loss: {epoch_loss:.4f}  Val Loss: {val_loss:.4f}")

torch.save(model.state_dict(), "mobilenetv2_chestxray.pth")